### Libaries

In [1]:
import robotic as ry
import numpy as np
import time
from scipy.interpolate import Rbf
from scipy.interpolate import CubicSpline
print('version:', ry.__version__, ry.compiled())

version: 0.1.3 compile time: Dec 15 2023 11:24:49


### Initalise configuration

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.view()

0

### Define color map

In [3]:
color_map = [[1,0,.5], # pink
             [0.5,1,0], # lime green
             [.5,1,1], # light turqois
             [0.5,0,1], # violet
             [1,0,0], # red
             [0,1,0], # green
             [0,0,1], # blue
             [1,.5,0], # orange
             [1,1,.5], # lime yellow
             [1,.5,1], # light pink                            
]

### Gripper functions

In [4]:
def gripper_open(C, bot, widht = 0.75, speed = 0.2):
    bot.gripperMove(ry._left, widht, speed)

    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)

def gripper_close(C, bot, widht = 0.01, speed = 0.2):
    bot.gripperMove(ry._left, widht, speed)

    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)

### Create N number of building cuboids

In [5]:
def create_n_boxes(C, N, position='fixed'):

    for i in range(N):
        box_name = 'box{}'.format(i + 1)

        if position == 'fixed':
            position_val1 = 0.1 * (i - 1)
            C.addFrame(box_name) \
                .setPosition([position_val1, 0.35, 0.70]) \
                .setShape(ry.ST.ssBox, size=[0.05, 0.05, 0.10, 0.005]) \
                .setColor(color_map[i % len(color_map)]) \
                .setContact(True) \
                .setMass(1e-2)


        elif position == 'random':
            C.addFrame(box_name) \
                .setPosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), 0.70]) \
                .setShape(ry.ST.ssBox, size=[0.05, 0.05, 0.10, 0.005]) \
                .setColor(color_map[i % len(color_map)]) \
                .setContact(True) \
                .setMass(1e-2)
    
    return C

### Define construction site

In [6]:
def create_construction_site(C, position = 'fixed'):
    
    if position == 'fixed':
        target_location = C.addFrame('construction_site', 'table')
        target_location.setShape(ry.ST.box, size = [.275, .275, .1])
        target_location.setRelativePosition([-.5, -.225, .0])
        target_location.setColor([1., 0.75, 0.])
        return target_location
    
    elif position == 'random':
        target_location = C.addFrame('construction_site', 'table')
        target_location.setShape(ry.ST.box, size = [.3, .3, .1])
        target_location.setRelativePosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), .0])
        target_location.setColor([1, 0.75, 0])
        return target_location
        

In [7]:
def create_pillar_target_postion(C, position = 'fixed', n_pillars = 2):
    
    if position == 'fixed':
        for i in range(0, n_pillars):
            target_pillar = C.addFrame('pillar_target_pos' + str(i+1), 'table')
            target_pillar.setShape(ry.ST.box, size=[.065, .065, .105])
            target_pillar.setRelativePosition([-.45 - (i / 10) , -.225, .0])
            target_pillar.setColor([0,1,0])
    
    elif position == 'random':
        for i in range(0, n_pillars):
            C = C.addFrame('pillar_target_pos' + str(i+1), 'table')
            target_pillar.setShape(ry.ST.box, size=[.065, .065, .105])
            target_pillar.setRelativePosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), .0])
            target_pillar.setColor([0,1,0])
        return C
        

### Define Komo problem

In [8]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C = create_n_boxes(C = C, N = 3, position = 'fixed')
goal = create_construction_site(C, position = 'fixed')
pillar_goal = create_pillar_target_postion(C, position = 'fixed')

C.view()

0

In [9]:
def gripper_to_block(C, bot, block = 'box1', prints = False):
    qHome = C.getJointState()

    komo = ry.KOMO(C, 1, 1, 0, False)

    komo.addObjective(
        times=[], 
        feature=ry.FS.jointState, 
        frames=[],
        type=ry.OT.sos, 
        scale=[1e-1], 
        target=qHome
    )
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', block], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', block], ry.OT.eq, [1e1], [-1])
    komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', block], ry.OT.eq, [1e1], [0])
    komo.addObjective([], ry.FS.distance, ['l_palm', block], ry.OT.ineq, [1e1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()

    bot.moveTo(q[-1])
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .5)

In [10]:
def gripper_to_block2(C, bot, block = 'box1', prints = False):

    komo = ry.KOMO(C, 1, 1, 0, False)

    #komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', block], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', block], ry.OT.eq, [1e1], [-1])
    komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', block], ry.OT.eq, [1e1], [0])
    komo.addObjective([], ry.FS.distance, ['l_palm', block], ry.OT.ineq, [1e1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()

    bot.moveTo(q[-1])
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .5)

In [11]:
def move_gripper_to(bot, add_frame_obj_str = 'target', offset = .075, prints=False):
    komo = ry.KOMO(C, 4, 1, 1, True)
    komo.addControlObjective([], 0, 1e-1)
    komo.addControlObjective([], 1, 1e0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1], [0])
    komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1], [0])
    
    komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e1])
    #komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.distance, ['l_palm', carry_item], ry.OT.ineq, [1e1])
    komo.addObjective([4], ry.FS.positionRel, [add_frame_obj_str, 'cameraWrist'], ry.OT.eq, [1.], [.0, .0, offset])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()
    
    bot.moveTo(q[-1])
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .1)

In [ ]:
def rotate_move_gripper_to(bot, add_frame_obj_str = 'target', offset = .075, prints=False):
    komo = ry.KOMO(C, 4, 1, 1, True)
    komo.addControlObjective([], 0, 1e-1)
    komo.addControlObjective([], 1, 1e0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1])
    komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1], [0])
    komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e-1], [0])
    
    komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e1])
    #komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', carry_item], ry.OT.eq, [1e1], [0])
    #komo.addObjective([], ry.FS.distance, ['l_palm', carry_item], ry.OT.ineq, [1e1])
    komo.addObjective([4], ry.FS.positionRel, [add_frame_obj_str, 'cameraWrist'], ry.OT.eq, [1.], [.0, .0, offset])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()
    
    bot.moveTo(q[-1])
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .1)


In [12]:
def generate_waypoint_guides(C, start='', target='', offset=0, n_waypoints=5, interpolation='linear'):
    start_frame = C.getFrame(start)
    start_pos = np.array(start_frame.getPosition())
    target_frame = C.getFrame(target)
    target_pos = np.array(target_frame.getPosition())
    target_pos[2] += offset


    if interpolation == 'linear':
        t_vals = np.linspace(0, 1, n_waypoints)
        x_spline = Rbf([0, 1], [start_pos[0], target_pos[0]], function='linear')
        y_spline = Rbf([0, 1], [start_pos[1], target_pos[1]], function='linear')
        z_spline = Rbf([0, 1], [start_pos[2], target_pos[2]], function='linear')

        positions = np.column_stack([x_spline(t_vals), y_spline(t_vals), z_spline(t_vals)])
    
    elif interpolation == 'cubic':
        t_vals = np.linspace(0, 1, n_waypoints)
        x_spline = Rbf([0, 1], [start_pos[0], target_pos[0]], function='linear')
        y_spline = Rbf([0, 1], [start_pos[1], target_pos[1]], function='linear')
        z_spline = Rbf([0, 1], [start_pos[2], target_pos[2]], function='cubic')
        
        positions = np.column_stack([x_spline(t_vals), y_spline(t_vals), z_spline(t_vals)])


    for i, pos in enumerate(positions):
        way_name = 'way' + str(i+1)
        C.addFrame(way_name).setShape(ry.ST.marker, [.1, .1, .1]).setPosition(pos).setColor([1, .5, 1])

    return C


In [13]:
C = generate_waypoint_guides(C, start = 'box1', target = 'pillar_target_pos2', offset=0.120, n_waypoints=5, interpolation='linear')
C.view()

0

In [14]:
bot = ry.BotOp(C, useRealRobot = False)

## First block

In [15]:
gripper_open(C, bot)
gripper_to_block(C, bot, block='box1')
gripper_close(C, bot)
move_gripper_to(bot, add_frame_obj_str = 'way1')
move_gripper_to(bot, add_frame_obj_str = 'way2')
move_gripper_to(bot, add_frame_obj_str = 'way3')
move_gripper_to(bot, add_frame_obj_str = 'way4')
move_gripper_to(bot, add_frame_obj_str = 'way5')
gripper_open(C, bot)
bot.home(C)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):222.046 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    125.963  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):     62.628  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    32.6945  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    14.0523  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    4.76188  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):   0.699383  ACCEPT
--newton-- it:   7  |Delta|:   0.115014  alpha:          1  evals:   8  f(y):   0.114352  ACCEPT
--newton-- it:   8  |Delta|:  0.0553725  alpha:          1  evals:   9  f(y):  0.0826405  ACCEPT
--newton-- it:   9  |Delta|:  0.0198054  alpha:          1  evals:  10  f(y

### Second Block

In [16]:
C = generate_waypoint_guides(C, start = 'box2', target = 'pillar_target_pos1', offset=0.120, n_waypoints=5, interpolation='linear')
bot = ry.BotOp(C, useRealRobot = False)
C.view()

-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:149(0) shutting down Simulation


0

In [17]:
gripper_to_block(C, bot, block='box2')
gripper_close(C, bot)
move_gripper_to(bot, add_frame_obj_str = 'way1')
move_gripper_to(bot, add_frame_obj_str = 'way2')
move_gripper_to(bot, add_frame_obj_str = 'way3')
move_gripper_to(bot, add_frame_obj_str = 'way4')
move_gripper_to(bot, add_frame_obj_str = 'way5')
gripper_open(C, bot)
bot.home(C)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):215.89 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    131.791  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    66.6626  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    36.8312  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):     18.165  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    6.50757  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    1.39933  ACCEPT
--newton-- it:   7  |Delta|:   0.144906  alpha:          1  evals:   8  f(y):   0.721747  ACCEPT
--newton-- it:   8  |Delta|:   0.170588  alpha:          1  evals:   9  f(y):   0.117179  ACCEPT
--newton-- it:   9  |Delta|:  0.0575916  alpha:          1  evals:  10  f(y)

### 3rd Block

In [19]:
box_1 = C.getFrame('box1')
box_2 = C.getFrame('box2')

box_pos1 = box_1.getPosition()
box_pos2 = box_2.getPosition()
offset = 0.075
middle_between_towers = np.mean([box_pos1, box_pos2], 0) + [0, 0, offset]
C.addFrame('block3_target').setShape(ry.ST.marker, [.1, .1, .1]).setPosition(middle_between_towers).setColor([1, .5, 1])
C.view()

[-0.5477072  -0.18103796  0.70139343] [-0.44434071 -0.17637043  0.70119691]


In [36]:
C = generate_waypoint_guides(C, start = 'box3', target = 'block3_target', offset=0.0125, n_waypoints=5, interpolation='linear')
bot = ry.BotOp(C, useRealRobot = False)
C.view()

-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:192(-1) frame already exists! returning existing without modifications!
-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:149(0) shutting down Simulation


37

### Clear config

In [87]:
C.clear()